In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import r2_score,mean_squared_error
import mlflow
from urllib.parse import urlparse 
from TimeSeriesForecast.utils.common import read_yaml,create_directories
from TimeSeriesForecast.constants import *
from dataclasses import dataclass
from pathlib import Path
import mlflow.sklearn
import dagshub
dagshub.init(repo_owner='carlosgutierrezch', repo_name='ml-flow', mlflow=True)
from notebooks.src.functions_torch import TimeSeries,eval_metrics
from torch.utils.data import DataLoader

Accessing as carlosgutierrezch

[2024-08-07 11:29:10,829:INFO:helpers:Accessing as carlosgutierrezch]


Initialized MLflow to track repo "carlosgutierrezch/ml-flow"

[2024-08-07 11:29:11,052:INFO:helpers:Initialized MLflow to track repo "carlosgutierrezch/ml-flow"]


Repository carlosgutierrezch/ml-flow initialized!

[2024-08-07 11:29:11,054:INFO:helpers:Repository carlosgutierrezch/ml-flow initialized!]


In [3]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: str
    mlflow_url: str
    batch_size: int
    num_epochs: int
    all_params: Path

In [4]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH,
                 schema_filepath= SCHEMA_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_eval_config(self)-> ModelEvaluationConfig:
        config= self.config.model_evaluation
        params= self.params.TorchModel

        return_eval= ModelEvaluationConfig(
                                            root_dir=config.root_dir,
                                            test_data_path= config.test_data_path,
                                            model_path= config.model_path,
                                            metric_file_name= config.metric_file_name,
                                            mlflow_url= config.mlflow_url,
                                            batch_size= params.batch_size,
                                            num_epochs= params.num_epochs,
                                            all_params= params
                                            )
        return return_eval

In [5]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config= config
        self.tensor=None
        self.X_test= None
        self.y_test=None
    def tensor_loader(self):
        try:
            self.tensor= torch.load(self.config.test_data_path)
            self.X_test,self.y_test= self.tensor['X_test'],self.tensor['y_test']
            self.test_dataset= TimeSeries(self.X_test,self.y_test)
            self.test_loader= DataLoader(self.test_dataset,batch_size=self.config.batch_size,shuffle=False)
            return self
        except Exception as e:
            raise e
    def model_loader(self):
        try:
            device= 'cuda:0' if torch.cuda.is_available() else 'cpu'
            model= torch.load(self.config.model_path)
            model.to(device)
            test_data= self.test_loader
            num_epochs= self.config.num_epochs
            with mlflow.start_run():
                model.eval()

                for epoch in range(num_epochs):
                    
                    for _,batch in enumerate(test_data):
                        x_batch,y_batch=batch[0].to(device),batch[1].to(device)
                        with torch.no_grad():
                            predicted = model(x_batch).to('cpu').numpy()
                            y_batch= y_batch.to('cpu').numpy()
                            (rmse,mae,r2)= eval_metrics(predicted,y_batch)
                        
                print(f'epoch: {epoch+1}/{num_epochs},rmse :{rmse}, mae: {mae},r2: {r2}') 
                                       
                mlflow.log_params(self.config.all_params)
                mlflow.log_metric('rmse', rmse)
                mlflow.log_metric('mae',mae)
                mlflow.log_metric('r2',r2)
        except Exception as e:
            raise e

In [6]:
try:
    configu= ConfigurationManager()
    model_eval= configu.get_eval_config()
    model_evaluation= ModelEvaluation(config=model_eval)
    model_evaluation.tensor_loader()
    model_evaluation.model_loader()
except Exception as e:
    raise e

[2024-08-07 11:29:12,219:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-08-07 11:29:12,250:INFO:common:yaml file: params.yaml loaded successfully]


[2024-08-07 11:29:12,266:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-08-07 11:29:12,272:INFO:common:created directory at: artifacts]
epoch: 10/10,rmse :0.2800808548927307, mae: 0.279976487159729,r2: -4582.60693359375
